# Contando Valores

Calcular el numero de datos disponibles para cada estacion y almacenar en una tabla el codigo de cada estacion junto con dicho valor

> [Parametros de los datos](#param)
>
> [Obtener los datos de Calidad del Aire](#getAQ)
>
> [Contar valores de Calidad del Aire](#countAQ)
>
> [Contar valores de Calidad del Aire](#countAQ)
>
> [Contar valores Meteorologicos](#countMto)
>
> [Agrupar todos los datos principales](#groupAll)

In [4]:
suppressMessages(library(saqgetr))
suppressMessages(library(lubridate))
suppressMessages(library(worldmet))

## Parametros de los datos <a id="param"></a>

In [5]:
# contaminantes a estudiar
pollutants <- c("no", "no2", "o3", "pm10")

# fechas de inicio y final de toma de datos
start_dt <- ymd_hms("2015-01-01 00:00:00")
end_dt <- ymd_hms("2020-10-01 00:00:00")

# fecha de inicio de confinamiento
lckdwn_strt <- ymd_hms("2020-03-14 00:00:00")

# Archivos de los datos
sitesAQ.fl = "../data/csv/sitesAQ.csv"                                           
dataAQ.fl = "../data/csv/dataAQ.csv"

sitesMto.fl = "../data/csv/sitesMto.csv"                                         
dataMto.fl = "../data/csv/dataMto.csv"                                          
years = 2010:2020,

final.fl = "../data/csv/nn_sites.csv"

# Guardar los datos de la calidad del aire
save.data = TRUE

## Obtener los datos de Calidad del Aire <a id="getAQ"></a>

In [ ]:
if (file.exists(sitesAQ.fl)) {
    sitesAQ <- read.csv(sitesAQ.fl, stringsAsFactors=T)[1:2]

    if (file.exists(dataAQ.fl)) {
        dataAQ <- read.csv(dataAQ.fl)
    } else {
        dataAQ <- get_saq_observations(site = levels(sitesAQ$site),
                                     variable = pollutants,
                                     valid_only = TRUE,
                                     start = start_dt,
                                     end = end_dt,
                                     verbose = TRUE
                                    )

    # Save data in a file for each station 
    if (save.data) {
            print("Saving Data...")
            write.csv(dataAQ, dataAQ.fl, row.names=FALSE)

            split.by.site(dataAQ, site.lv="all",
                          folder="../data/csv/dataAQ/")
        }
    }
}

In [ ]:
head(dataAQ)

## Contar valores de Calidad del Aire <a id="countAQ"></a>

In [ ]:
numCount <- c()
for (st in levels(sitesAQ$site)) {
    numCount <- c(numCount, nrow(dataAQ[dataAQ$site == st, ]))
}

nn.sitesAQ <- data.frame(siteAQ = levels(sitesAQ$site),
                         countAQ = numCount)
rm(dataAQ, numCount)

In [ ]:
head(nn.sites)

## Obtener los datos Meteorologicos <a id="getMto"></a>

In [ ]:
if (file.exists(sitesMto.fl)) {
    sitesMto <- read.csv(sitesMto.fl, stringsAsFactors=T)
    
    if (file.exists(dataMto.fl)) {
        dataMto <- read.csv(dataMto.fl)
    } else {
        # Get Data from worlmet
        dataMto <- importNOAA(code = levels(sitesMto$code),
                           year = years,
                           hourly = FALSE,
                           n.cores = 4,
                           quiet = FALSE,
                           path = NA
                          )
    
        # Save data in a file for each station 
        if (save.data) {
            print("Saving Data...")
            write.csv(dataMto, dataMto.fl, row.names=FALSE)

            split.by.site(dataMto, site.lv="all",
                          folder="../data/csv/dataMto/")
        }
    }
}

## Contar valores Meteorologicos <a id="countMto"></a>

In [ ]:
countMto <- NA                                                                                          
codeMto <- NA
code.lv <- levels(as.factor(dataMto$code))

for (i in 1:length(code.lv)) {
    codeMto <- rbind(codeMto, code.lv[i])
    
    countMto <- rbind(countMto, colSums(!is.na(
                                        dataMto[dataMto$code == code.lv[i],
                                        !colnames(dataMto) %in% c("date",
                                                               "station",
                                                               "latitude",
                                                               "longitude",
                                                               "elev"
                                                              )])
                                        )
                                / nrow(dataMto[dataMto$code ==code.lv[i], ])
                     )
    countMto[i+1, "code"] <- nrow(dataMto[dataMto$code == code.lv[i], ])
}
countMto <- data.frame(countMto[complete.cases(countMto), ]) %>%
                rename(countMto = code)

codeMto <- data.frame(codeMto[complete.cases(codeMto), ])
colnames(codeMto) <- c("siteMto")

countMto <- cbind(siteMto, countMto)

#final.sites.mto
nn.sitesMto <- merge(x = sitesMto %>%
                        rename(siteMto=code) %>%
                        select(siteMto, dist, siteAQ),
                     y = countMto,
                     by = "siteMto", all=TRUE
                    )
rm(dataMto, countMto, sitesMto, codeMto)

In [ ]:
head(nn.sitesMto)

## Agrupar todos los datos principales <a id="groupAll"></a>

In [ ]:
nn.sites <- merge(x = nn.sitesAQ,
                  y = nn.sitesMto,
                  by = "siteAQ", all = TRUE)

In [ ]:
write.csv(nn.sites, final.fl, row.names=FALSE)